# Data visualization using rerun
https://github.com/rerun-io/pi0-lerobot/blob/main/notebooks/01_load_lerobot_dataset.ipynb


In [2]:
from lerobot.common.datasets.lerobot_dataset import (
    LeRobotDataset,
    LeRobotDatasetMetadata,
)
from beartype.door import die_if_unbearable

sample_number:int = 10
die_if_unbearable(sample_number, int) # should not raise an exception

sample_text: int = "hello" # doesn't raise an exception, but should
try:
    die_if_unbearable(sample_text, int) # will raise an exception since the type is not int
except Exception as e:
    print(e)

Die_if_unbearable() value 'hello' violates type hint <class 'int'>, as str 'hello' not instance of int.


In [3]:
dataset_idx:int = 5
# die_if_unbearable(dataset_idx, int)

# repo_id:str = "lerobot/aloha_static_coffee_new" #available_datasets[dataset_idx]
# repo_id:str =  available_datasets[dataset_idx]
repo_id:str = "DanqingZ/so100_test_6"

print(repo_id)

# die_if_unbearable(repo_id, str)

# We can have a look and fetch its metadata to know more about it:
ds_meta = LeRobotDatasetMetadata(repo_id)

# By instantiating just this class, you can quickly access useful information about the content and the
# structure of the dataset without downloading the actual data yet (only metadata files — which are
# lightweight).
total_episodes: int = ds_meta.total_episodes
print(f"Total number of episodes: {total_episodes}")
avg_frames_per_episode: float = ds_meta.total_frames / total_episodes
print(f"Average number of frames per episode: {avg_frames_per_episode:.3f}")
fps: int = ds_meta.fps
print(f"Frames per second used during data collection: {fps}")
robot_type: str = ds_meta.robot_type
print(f"Robot type: {robot_type}")
camera_keys: list[str] = ds_meta.camera_keys
print(f"keys to access images from cameras: {camera_keys=}\n")

print(ds_meta)

DanqingZ/so100_test_6
Total number of episodes: 2
Average number of frames per episode: 447.000
Frames per second used during data collection: 30
Robot type: so100
keys to access images from cameras: camera_keys=['observation.images.laptop', 'observation.images.phone']

LeRobotDatasetMetadata({
    Repository ID: 'DanqingZ/so100_test_6',
    Total episodes: '2',
    Total frames: '894',
    Features: '['action', 'observation.state', 'observation.images.laptop', 'observation.images.phone', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index']',
})',



In [4]:
from pprint import pprint
from typing import Any, Literal, NewType, TypedDict

class BaseFeature(TypedDict, total=False):
    """
    For dtype in {"float32", "int64", "bool", "string"}.
    'shape' is a tuple of ints, 'names' can be None, a list, or a dict.
    """
    dtype: Literal["float32", "int64", "bool", "string"]
    shape: tuple[int, ...]
    names: list[str] | dict[str, list[str]] | None

class VideoFeature(TypedDict, total=False):
    """
    For dtype == "video".
    Must include 'video_info'.
    """
    dtype: Literal["video"]
    shape: tuple[int, ...]
    names: list[str] | dict[str, list[str]] | None
    video_info: dict[str, Any]


Feature = BaseFeature | VideoFeature
FeatureName = NewType('FeatureName', str)
FeaturesDict = dict[FeatureName, Feature]

features:FeaturesDict= ds_meta.features
die_if_unbearable(features, FeaturesDict)
print("Features:\n")
pprint(features)

Features:

{'action': {'dtype': 'float32',
            'names': ['main_shoulder_pan',
                      'main_shoulder_lift',
                      'main_elbow_flex',
                      'main_wrist_flex',
                      'main_wrist_roll',
                      'main_gripper'],
            'shape': (6,)},
 'episode_index': {'dtype': 'int64', 'names': None, 'shape': (1,)},
 'frame_index': {'dtype': 'int64', 'names': None, 'shape': (1,)},
 'index': {'dtype': 'int64', 'names': None, 'shape': (1,)},
 'observation.images.laptop': {'dtype': 'video',
                               'info': {'has_audio': False,
                                        'video.channels': 3,
                                        'video.codec': 'av1',
                                        'video.fps': 30,
                                        'video.height': 480,
                                        'video.is_depth_map': False,
                                        'video.pix_fmt': 'yuv420p',

In [5]:
# You can then load the actual dataset from the hub.
# Either load any subset of episodes:
episodes: list[int] = [0, 1]
die_if_unbearable(episodes, list[int])
dataset = LeRobotDataset(repo_id, episodes=episodes)

# And see how many frames you have:
print(f"Selected episodes: {dataset.episodes}")
print(f"Number of episodes selected: {dataset.num_episodes}")
print(f"Number of frames selected: {dataset.num_frames}")

# Or simply load the entire dataset:
full_dataset = LeRobotDataset(repo_id)
print(f"Number of episodes selected: {full_dataset.num_episodes}")
print(f"Number of frames selected: {full_dataset.num_frames}")

# The previous metadata class is contained in the 'meta' attribute of the dataset:
print(full_dataset.meta)

# LeRobotDataset actually wraps an underlying Hugging Face dataset
# (see https://huggingface.co/docs/datasets for more information).
print(full_dataset.hf_dataset)

Selected episodes: [0, 1]
Number of episodes selected: 2
Number of frames selected: 894
Number of episodes selected: 2
Number of frames selected: 894
LeRobotDatasetMetadata({
    Repository ID: 'DanqingZ/so100_test_6',
    Total episodes: '2',
    Total frames: '894',
    Features: '['action', 'observation.state', 'observation.images.laptop', 'observation.images.phone', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index']',
})',

Dataset({
    features: ['action', 'observation.state', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index'],
    num_rows: 894
})


In [6]:
dataset.features.keys()

dict_keys(['action', 'observation.state', 'observation.images.laptop', 'observation.images.phone', 'timestamp', 'frame_index', 'episode_index', 'index', 'task_index'])

In [7]:
dataset.features["action"]

{'dtype': 'float32',
 'shape': (6,),
 'names': ['main_shoulder_pan',
  'main_shoulder_lift',
  'main_elbow_flex',
  'main_wrist_flex',
  'main_wrist_roll',
  'main_gripper']}

## 1. Extract task name

In [18]:
len(dataset)

894

In [19]:
i=0
print(dataset[i]['task'])

Grasp a lego block and put it in the bin.


## 2. Image

In [8]:
from jaxtyping import Float32, UInt8
import torch
from einops import rearrange
import numpy as np
import rerun as rr

episode_index:int = 0
die_if_unbearable(episode_index, int)

from_idx:int = dataset.episode_data_index["from"][episode_index].item()
to_idx:int = dataset.episode_data_index["to"][episode_index].item()
print(f"Episode {episode_index} starts at frame {from_idx} and ends at frame {to_idx}")
# # Then we grab all the image frames from the first camera:
camera_key: str = dataset.meta.camera_keys[0]
die_if_unbearable(camera_key, str)

# loading all frames like this can take a while, so only do the first 10 frames
frames:list[Float32[torch.Tensor, "3 H W"]] = [dataset[idx][camera_key] for idx in range(from_idx, 150)]
die_if_unbearable(frames, list[Float32[torch.Tensor, "3 H W"]])

# The show that the frames are float32 tensors with shape (3, H, W) or in this case (3, 480, 640)
print(type(frames[0]))
print(frames[0].shape)
print(frames[0].dtype)

# lets visualize these images

rr.init("lerobot images")
for idx, frame in enumerate(frames):
    #rr.set_time_sequence("frame_idx", idx)
    rr.set_time("foo", timestamp=idx)
    # convert to h w 3
    rgb_tensor:Float32[torch.Tensor, "H W 3"] = rearrange(frame, 'C H W -> H W C')
    die_if_unbearable(rgb_tensor, Float32[torch.Tensor, "H W 3"])

    rgb_array:Float32[np.ndarray, "H W 3"] = rgb_tensor.numpy(force=True)
    die_if_unbearable(rgb_array, Float32[np.ndarray, "H W 3"])

    # convert from 0-1 to 0-255 and convert to uint8
    rgb_array:UInt8[np.ndarray, "H W 3"] = (rgb_array * 255).astype(np.uint8)
    die_if_unbearable(rgb_array, UInt8[np.ndarray, "H W 3"])

    rr.log("image", rr.Image(rgb_array).compress(jpeg_quality=70))

rr.notebook_show()

Episode 0 starts at frame 0 and ends at frame 447


objc[38096]: Class AVFFrameReceiver is implemented in both /opt/anaconda3/envs/lerobot/lib/python3.10/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x11eea83a8) and /opt/anaconda3/envs/lerobot/lib/libavdevice.61.3.100.dylib (0x176280848). One of the two will be used. Which one is undefined.
objc[38096]: Class AVFAudioReceiver is implemented in both /opt/anaconda3/envs/lerobot/lib/python3.10/site-packages/av/.dylibs/libavdevice.61.3.100.dylib (0x11eea83f8) and /opt/anaconda3/envs/lerobot/lib/libavdevice.61.3.100.dylib (0x176280898). One of the two will be used. Which one is undefined.


<class 'torch.Tensor'>
torch.Size([3, 480, 640])
torch.float32


Viewer()

In [9]:
import rerun.blueprint as rrb
import tqdm
from jaxtyping import Int64
from lerobot.scripts.visualize_dataset import EpisodeSampler

rr.init("Final notebook visualization")

dataset = LeRobotDataset(repo_id)

episode_index:int = 0
episode_sampler = EpisodeSampler(dataset, episode_index)

batch_size:int = 1
dataloader = torch.utils.data.DataLoader(
    dataset,
    num_workers=0,
    batch_size=batch_size,
    sampler=episode_sampler,
)

blueprint =rrb.Blueprint(
    rrb.Vertical(
        rrb.Grid(contents=[rrb.Spatial2DView(origin=key) for key in dataset.meta.camera_keys]),
        rrb.TimeSeriesView(),
    ),
    collapse_panels=True
)
# rr.log("test", rr.Scalar(0.0))
# rr.notebook_show(blueprint=blueprint, height=500,width=1000)
# for batch in tqdm.tqdm(dataloader, total=len(dataloader)):
#     # iterate over the batch
#     batch_index:Int64[torch.Tensor, "batch_size"] = batch["index"]
#     die_if_unbearable(batch_index, Int64[torch.Tensor, "batch_size"])
#     for i in range(len(batch_index)):
#         frame_index:int = batch["frame_index"][i].item()
#         die_if_unbearable(frame_index, int)
#         rr.set_time_sequence("frame_index", frame_index)

#         timestamp:float = batch["timestamp"][i].item()
#         die_if_unbearable(timestamp, float)
#         rr.set_time_seconds("timestamp", timestamp)
    
#         # display each camera image
#         for key in dataset.meta.camera_keys:
#             # convert from tensor format to numpy
#             rgb_tensor:Float32[torch.Tensor, "3 H W"] = batch[key][i]
#             rgb_array:Float32[np.ndarray, "H W 3"] = rearrange(rgb_tensor, 'C H W -> H W C').numpy(force=True)
#             rgb_array:UInt8[np.ndarray, "H W 3"] = (rgb_array * 255).astype(np.uint8)
#             rr.log(
#                 key,
#                 rr.Image(rgb_array).compress(
#                     jpeg_quality=95
#                 ),
#             )
    
#         # display each dimension of action space (e.g. actuators command)
#         if "action" in batch:
#             for dim_idx, val in enumerate(batch["action"][i]):
#                 rr.log(f"action/{dim_idx}", rr.Scalar(val.item()))
    
#         # display each dimension of observed state space (e.g. agent position in joint space)
#         if "observation.state" in batch:
#             for dim_idx, val in enumerate(batch["observation.state"][i]):
#                 rr.log(f"state/{dim_idx}", rr.Scalar(val.item()))
    
#         if "next.done" in batch:
#             rr.log("next.done", rr.Scalar(batch["next.done"][i].item()))
    
#         if "next.reward" in batch:
#             rr.log("next.reward", rr.Scalar(batch["next.reward"][i].item()))
    
#         if "next.success" in batch:
#             rr.log("next.success", rr.Scalar(batch["next.success"][i].item()))
            

In [10]:
for batch in tqdm.tqdm(dataloader, total=len(dataloader)):
    # iterate over the batch
    print(batch['task'])

  4%|████▏                                                                                                   | 18/447 [00:00<00:04, 89.88it/s]

['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']


  6%|██████▎                                                                                                 | 27/447 [00:00<00:05, 79.62it/s]

['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']


 10%|██████████▋                                                                                             | 46/447 [00:00<00:04, 86.40it/s]

...

['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in

 98%|████████████████████████████████████████████████████████████████████████████████████████████████████▏ | 439/447 [00:04<00:00, 108.95it/s]

['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 447/447 [00:04<00:00, 106.21it/s]

['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']
['Grasp a lego block and put it in the bin.']


In [12]:
len(dataset)

894